# Pregunta 2

## Clase receiver

Para implementar esta clase usamos la informacion de clases junto con el "molde" que nos entrega el enunciado

```
class RSAReceiver:
    
    def __init__(self, bit_len: int) -> None:
        pass
    
    def get_public_key(self) -> bytearray:
        pass
    
    def decrypt(self, ciphertext: bytearray) -> str:
        pass
```

### Número Primo al azar

Como para generar una llave pública se requiere seguir los pasos de RSA hacen falta algunas funciones auxiliares como por ejemplo: **conseguir un número primo al azar**
Para esto se puede usar una función creada en base a lo visto en clases

In [592]:
import random
import math

def _es_potencia(n): # no es lo mas elegante
    f = math.log(n) / math.log(2)
    if ((f - int(f)) == 0.0):
        return True
    for x in range(3, 100000, 2):
        f = math.log(n) / math.log(x)
        if ((f - int(f)) == 0.0):
            return True

def _test_primalidad(n: int, k:int) -> bool:
    if n == 1:
        return False
    elif n == 2:
        return True
    elif n%2 == 0:
        return False
    elif _es_potencia(n):
        return False
    else:
        neg = 0
        for i in range(1, k+1):
            a = random.randint(2, n-1)
            if math.gcd(a,n) > 1:
                return False
            else:
                b = pow(a,(n-1)//2,n)
                if b == n-1:
                    neg = neg + 1
                elif b!=1:
                    return False
        if neg > 0:
            return True
        else:
            return False

Debido a que esto no es perfecto, ya que mi funcion _es_potencia no toma todas las posibles potencias, por lo que tiene sentido usar :
```

from Crypto.Util import number
number.getPrime(n)
```
donde n es un número de bits

In [593]:
# from Crypto.Util import number
# import time
# time.clock = time.time # sin esta linea hay un error por un tema de version de python
# number.getPrime(1000)

8764570060165486689952504683919851099746096356548249807911082950212826843930744567496956959491551045730240522183539566420620352743302376744424536640071164891347895091129416145042627736557010052624168207783282318967215540264908182574311358638780496463169698720168313070326082672472212383937061116629109

In [594]:
# _test_primalidad(number.getPrime(1000), 20)

True

Si quisiera usar el metodo aleatorio tendria que hacer algo como lo siguiente

In [595]:
def _random_big_prime(bit_len):
    yes = False
    
    while not yes:
        n = random.getrandbits(bit_len)
        if _test_primalidad(n, 20):
            yes = True
    
    return n

In [596]:
# _random_big_prime(10)

2

Bueno, se ve que funciona.

### Calcular inverso modular
Se puede hacer usando el metodo de euclides, pero desde python 3.8 existe pow, que trabaja justamente con modulos y acepta -1 como potencia, por ejemplo:

In [597]:
# pow(33, -1, 140)

17

Que es justamente el ejemplo que se dio en clases

### Generar Coprimo
Se usa el ejemplo de las diapositivas

In [598]:
def _gen_primo_relativo(n):
    while True:
        a = random.randint(0, n)
        if math.gcd(a,n)==1:
            return a
            

In [599]:
# _gen_primo_relativo(100)

97

### Ahora la definicion de la clase

In [600]:
class RSAReceiver:

    def __init__(self, bit_len: int) -> None:
        self.bit_len = bit_len
        
        P = _random_big_prime(self.bit_len)
        Q = _random_big_prime(self.bit_len)
        
        N = P*Q
        #print("N: "+str(N))
        Phi = (P-1)*(Q-1)
        
        d = _gen_primo_relativo(Phi)
        e = pow(d, -1, Phi)
        
        #print("d: "+str(d))
        #print("e: "+str(e))
        
        self.public_key = (e,N)
        self.secret_key = (d,N)

    def get_public_key(self) -> bytearray:
        e = self.public_key[0]
        N = self.public_key[1]
        
        # Para el tamaño en bytes (no bits)
        e_bytes = (e.bit_length() + 7) // 8
        N_bytes = (N.bit_length() + 7) // 8
        
        b_e = e.to_bytes(e_bytes,'big')
        b_N = N.to_bytes(N_bytes,'big')
        
        return e_bytes.to_bytes(4,'big') + b_e + N_bytes.to_bytes(4,'big') + b_N 

    def decrypt(self, ciphertext: bytearray) -> str:
        d = self.secret_key[0]
        N = self.secret_key[1]
        
        # necesito el n para separar en bloques
        n = (N.bit_length()-1)//8 
        
        decriptado = ""
        
        for i in range(0, len(ciphertext), n+1):
            c = ciphertext[i:i+n+1]
            
            int_c = int.from_bytes(c, "big")
            
            n_dec = pow(int_c, d, N)
            
            dec = n_dec.to_bytes(n+1, "big").strip(b'\x00')
            
            decriptado += dec.decode("utf-8")
            
        return decriptado
            

In [601]:
#a = RSAReceiver(100)

In [602]:
#int.from_bytes(a.get_public_key()[:4], 'big') # 25
#int.from_bytes(a.get_public_key()[4:4+25], 'big')

In [603]:
#((329861860025362046740653651111990613030854999475473815122321).bit_length() + 7)//8

## Clase Sender

Ahora hay que hacer la clase sender, que es considerablemente más sencilla, ya que no hay que generar llaves

In [604]:
class RSASender:
    
    def __init__(self, public_key: bytearray) -> None:
        n_e = int.from_bytes(public_key[:4], "big")
        self.e = int.from_bytes(public_key[4:4+n_e], "big")
        n_N = int.from_bytes(public_key[4+n_e:8+n_e], "big")
        self.N = int.from_bytes(public_key[8+n_e:8+n_e+n_N], "big")
        
        #print("N: "+str(self.N))
        #print("e: "+str(self.e))
            
    def encrypt(self, message: str) -> bytearray:
        # Primero de mensaje a bytes
        b_mensaje = bytearray(message, 'utf-8')
        
        n = (self.N.bit_length()-1)//8 
        
        encriptado = bytearray(b'')
        
        # Avanzo bloque por bloque con un indice
        for i in range(0, len(b_mensaje), n):
            b = b_mensaje[i:i+n]
            
            a = int.from_bytes(b, 'big')
            c = pow(a, self.e, self.N)
            
            # Una vez encriptado se pasa a bytes y se agrega al bytearray de largo n+1
            encriptado += c.to_bytes(n+1, "big")
        return encriptado
            
                
        

In [605]:
#a = RSAReceiver(100)
#s = RSASender(a.get_public_key())

In [607]:
# m = (
#     'Being open source means anyone can independently review '
#     'the code. If it was closed source, nobody could verify the '
#     'security. I think it’s essential for a program of this '
#     'nature to be open source.'
# )
# enc = s.encrypt(m)
# a.decrypt(enc)==m

True